Корпус представляет собой несколько таблиц с данными о последовательсти реплик в диалогах, названиях фильмов и т.д. Для выполнения следующих задач вам необходимо объединить таблицы в одну с помощью pandas. В таблице должна быть представлена следующая информация: диалог (список реплик), спикеры (список), фильм, из которого взят диалог, жанры фильма.

In [1]:
import pandas as pd

In [76]:
columns_mtm = ['movieID', 'movie title', 'movie year', 'IMDB rating', 'no. IMDB votes', 'genres']
df_movie_titles_metadata = pd.read_csv('../data/raw_data/movie_titles_metadata.tsv', sep='\t', header=None, 
                                       names=columns_mtm)
# df_movie_titles_metadata = pd.read_table('../data/raw_data/movie_titles_metadata.tsv')
df_movie_titles_metadata.movieID.value_counts()

movieID
m0      1
m414    1
m407    1
m408    1
m409    1
       ..
m207    1
m208    1
m209    1
m210    1
m616    1
Name: count, Length: 617, dtype: int64

In [66]:
columns_mcm = ['characterID', 'character name', 'movieID', 'movie title', 'gender', 'position']
df_movie_characters_metadata = pd.read_csv('../data/raw_data/movie_characters_metadata.tsv', sep='\t', header=None, 
                                       names=columns_mcm)
df_movie_characters_metadata.head()

,characterID,character name,movieID,movie title,gender,position
0,u0,BIANCA,m0,10 things i hate about you,f,4
1,u1,BRUCE,m0,10 things i hate about you,?,?
2,u2,CAMERON,m0,10 things i hate about you,m,3
3,u3,CHASTITY,m0,10 things i hate about you,?,?
4,u4,JOEY,m0,10 things i hate about you,m,6


In [5]:
columns_mc = ['character1ID', 'character2ID', 'movieID', 'list']
df_movie_conversations = pd.read_csv('../data/raw_data/movie_conversations.tsv', sep='\t', header=None, 
                                       names=columns_mc)
df_movie_conversations.head()


,character1ID,character2ID,movieID,list
0,u0,u2,m0,['L194' 'L195' 'L196' 'L197']
1,u0,u2,m0,['L198' 'L199']
2,u0,u2,m0,['L200' 'L201' 'L202' 'L203']
3,u0,u2,m0,['L204' 'L205' 'L206']
4,u0,u2,m0,['L207' 'L208']


In [9]:
columns_ml = ['lineID', 'characterID', 'movieID', 'character_name', 'text']
df_movie_lines = pd.read_csv('../data/raw_data/movie_lines.tsv', sep='\t', header=None, 
                            names=columns_ml, on_bad_lines='skip') # там немного кривых строк
df_movie_lines.head()
df_movie_lines['lineID'].value_counts()

lineID
L1045      1
L305579    1
L305484    1
L305485    1
L305486    1
          ..
L542620    1
L542619    1
L542618    1
L542590    1
L666256    1
Name: count, Length: 293210, dtype: int64

### Итоговые таблицы

In [63]:
df_movie_titles_metadata = df_movie_titles_metadata.drop(['movie year', 'IMDB rating', 'no. IMDB votes'], axis=1)
df_movie_titles_metadata.to_csv('../data/clean_data/movie_titles_metadata.csv', index=False)
df_movie_titles_metadata.head()

,movieID,movie title,genres
0,m0,10 things i hate about you,['comedy' 'romance']
1,m1,1492: conquest of paradise,['adventure' 'biography' 'drama' 'history']
2,m2,15 minutes,['action' 'crime' 'drama' 'thriller']
3,m3,2001: a space odyssey,['adventure' 'mystery' 'sci-fi']
4,m4,48 hrs.,['action' 'comedy' 'crime' 'drama' 'thriller']


In [70]:
df_movie_characters_metadata = df_movie_characters_metadata.drop(['movie title', 'gender', 'position'], axis=1)
df_movie_characters_metadata.to_csv('../data/clean_data/movie_characters_metadata.csv', index=False)
df_movie_characters_metadata.head()

,characterID,character name,movieID
0,u0,BIANCA,m0
1,u1,BRUCE,m0
2,u2,CAMERON,m0
3,u3,CHASTITY,m0
4,u4,JOEY,m0


In [10]:

# df_movie_lines = df_movie_lines.drop(['movie title', 'gender'], axis=1)
df_movie_lines.to_csv('../data/clean_data/movie_lines.csv', index=False)
df_movie_lines.head()

,lineID,characterID,movieID,character_name,text
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.


In [6]:
df_movie_conversations = df_movie_conversations.drop(['character2ID'], axis=1)
df_movie_conversations.to_csv('../data/clean_data/movie_conversations.csv', index=False)
df_movie_conversations.head()

,character1ID,movieID,list
0,u0,m0,['L194' 'L195' 'L196' 'L197']
1,u0,m0,['L198' 'L199']
2,u0,m0,['L200' 'L201' 'L202' 'L203']
3,u0,m0,['L204' 'L205' 'L206']
4,u0,m0,['L207' 'L208']


## Мердж 

In [81]:
import pandas as pd
df_characters = pd.read_csv('../data/clean_data/movie_characters_metadata.csv')
df_conversations = pd.read_csv('../data/clean_data/movie_conversations.csv')
df_lines = pd.read_csv('../data/clean_data/movie_lines.csv')
df_titles = pd.read_csv('../data/clean_data/movie_titles_metadata.csv')

In [146]:
# вставляю строки вместо имен
def lines2list(line_list):    
    itog = []
    line_list = line_list.replace(' ', ', ')
    line_list = re.sub("[^A-Za-z0-9,]", "", line_list)
    line = line_list.split(sep=',')
    for i in range(len(line)):
        filter = df_lines['lineID'] == line[i]
        text = df_lines.loc[filter]['text'][0]
        print(text)
        itog = ",".join(text)
    return itog

df_lines = pd.read_csv('../data/clean_data/movie_lines.csv')
df_lines = df_lines[:10]
df_conversations['text_full'] = df_conversations['list'].apply(lines2list)
df_conversations.head()

KeyError: 0

In [147]:
import re 

i = df_conversations['list'][0]
i =  i.replace(' ', ', ')

# i =  i.replace("]'","")
# i =  i.replace("'[","")
i = re.sub("[^A-Za-z0-9,]", "", i)
i = i.split(sep=',')
print(i[0])
# print(set(i))

L194
